# gpu/ram/drive

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 25 09:37:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
import gc
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

Mounted at /content/drive


In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# import

In [7]:
import random
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [46]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 92.4 MB/s 


# modeling

In [5]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x['obs_time'] = x['obs_time'].str[:2]
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]
            time_list = day['obs_time'].unique()
            if len(time_list) > 24 :
                for i in range(0,len(time_list)):
                    x['obs_time'] = x['obs_time'].replace(time_list[24+i],time_list[i])
            for col in col_list :
                for time in time_list :
                    value = day[day['obs_time']==time][col].iloc[0]
                    x[col+time] = value
            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)
    return train

In [17]:
train_input_path, train_target_path = 'drive/MyDrive/dacon/data/train_input/*.csv', 'drive/MyDrive/dacon/data/train_target/*.csv'
test_input_path, test_target_path = 'drive/MyDrive/dacon/data/test_input/*.csv', 'drive/MyDrive/dacon/data/test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [18]:
train_test = train.copy()
test_test = test.copy()

In [47]:
import catboost

In [ ]:
cat = catboost.CatBoostRegressor(task_type='GPU')
cat.fit(X, y)
y_pred = cat.predict(testX)

In [57]:
all_target_list = sorted(glob.glob(test_target_path))

In [58]:
i=0
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = y_pred[i*28:i*28+28]
    submit_df.to_csv(test_path, index=False)
    i+=1

In [55]:
X = train_test.iloc[:,:-1]
y = train_test.iloc[:,-1:]
testX = test_test.iloc[:,:-1]
testy = test_test.iloc[:,-1:]